In [25]:
import sys
import os

# Add the current project directory to Python path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print(f"Working directory: {current_dir}")
print("Available folders:", [f for f in os.listdir('.') if os.path.isdir(f)])

Working directory: C:\Users\Alqama\quant_trading_engine
Available folders: ['.ipynb_checkpoints', 'engine', 'risk_management', 'strategies']


In [27]:
import yfinance as yf
import pandas as pd
import numpy as np

In [17]:
import os
import sys

print("Current working directory:", os.getcwd())
print("\nContents of current directory:")
for item in os.listdir('.'):
    if os.path.isdir(item):
        print(f"📁 {item}/")
        # Show contents of each subdirectory
        try:
            sub_items = os.listdir(item)
            for sub_item in sub_items:
                print(f"   📄 {sub_item}")
        except:
            pass
    else:
        print(f"📄 {item}")

print(f"\nPython path includes:")
for path in sys.path:
    print(f"  {path}")

Current working directory: C:\Users\Alqama\quant_trading_engine

Contents of current directory:
📁 .ipynb_checkpoints/
   📄 trading_backtest-checkpoint.ipynb
📁 engine/
   📄 .ipynb_checkpoints
   📄 backtest.py
   📄 __init__.py
   📄 __pycache__
📁 risk_management/
   📄 .ipynb_checkpoints
   📄 drawdown.py
   📄 position_sizing.py
   📄 __init__.py
📁 strategies/
   📄 .ipynb_checkpoints
   📄 mean_reversion.py
   📄 momentum_strategy.py
   📄 pairs_trading_strategy.py
   📄 __init__.py
📄 trading_backtest.ipynb

Python path includes:
  C:\Users\Alqama\quant_trading_engine
  C:\Users\Alqama\anaconda3\python312.zip
  C:\Users\Alqama\anaconda3\DLLs
  C:\Users\Alqama\anaconda3\Lib
  C:\Users\Alqama\anaconda3
  
  C:\Users\Alqama\anaconda3\Lib\site-packages
  C:\Users\Alqama\anaconda3\Lib\site-packages\win32
  C:\Users\Alqama\anaconda3\Lib\site-packages\win32\lib
  C:\Users\Alqama\anaconda3\Lib\site-packages\Pythonwin


In [23]:
try:
    with open('engine/backtest.py', 'r') as f:
        content = f.read()
        if 'class BacktestEngine' in content:
            print("✅ BacktestEngine class found in engine/backtest.py")
        else:
            print("❌ BacktestEngine class NOT found in engine/backtest.py")
            print("First 500 characters of file:")
            print(content[:500])
except Exception as e:
    print(f"❌ Error reading engine/backtest.py: {e}")
try:
    with open('strategies/momentum.py', 'r') as f:
        content = f.read()
        if 'class MomentumStrategy' in content:
            print("✅ MomentumStrategy class found in strategies/momentum.py")
        else:
            print("❌ MomentumStrategy class NOT found in strategies/momentum.py")
            print("First 500 characters of file:")
            print(content[:500])
except Exception as e:
    print(f"❌ Error reading strategies/momentum.py: {e}")

❌ BacktestEngine class NOT found in engine/backtest.py
First 500 characters of file:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "6416af59-0c99-4ea8-9667-cf41c923a6d0",
   "metadata": {},
   "outputs": [
    {
     "ename": "ModuleNotFoundError",
     "evalue": "No module named 'risk_management'",
     "output_type": "error",
     "traceback": [
      "\u001b---------------------------------------------------------------------------\u001b",
      "\u001bModuleNotFoundError\u001b                       Traceback (most recent call l
❌ Error reading strategies/momentum.py: [Errno 2] No such file or directory: 'strategies/momentum.py'


In [29]:
import yfinance as yf

class BacktestEngine:
    def __init__(self, ticker, start_date, end_date, strategy, initial_cash=100_000):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.strategy = strategy
        self.initial_cash = initial_cash
        self.results = None
        self._prepare_data()

    def _prepare_data(self):
        df = yf.download(self.ticker, start=self.start_date, end=self.end_date)[['Adj Close']].copy()
        df.columns = ['price']
        df.dropna(inplace=True)

        df = self.strategy.generate_signals(df)
        df['returns'] = df['price'].pct_change()
        df['strategy'] = df['position'].shift(1) * df['returns']
        df.dropna(inplace=True)

        df['equity_curve'] = self.initial_cash * (1 + df['strategy']).cumprod()
        df['buy_hold'] = self.initial_cash * (1 + df['returns']).cumprod()
        self.results = df

    def plot_performance(self):
        if self.results is not None:
            self.results[['equity_curve', 'buy_hold']].plot(figsize=(12, 6), title=f"{self.ticker} Strategy Backtest")